# Exercise 1: t-SNE

## Do not start the exercise until you fully understand the submission guidelines.


* The homework assignments are executed automatically.
* Failure to comply with the following instructions will result in a significant penalty.
* Appeals regarding your failure to read these instructions will be denied.

## Read the following instructions carefully:

1. This Jupyter notebook contains all the step-by-step instructions needed for this exercise.
1. Write **efficient**, **vectorized** code whenever possible. Some calculations in this exercise may take several minutes when implemented efficiently, and might take much longer otherwise. Unnecessary loops will result in point deductions.
1. You are responsible for the correctness of your code and should add as many tests as you see fit to this jupyter notebook. Tests will not be graded nor checked.
1. You are allowed to use functions and methods from the [Python Standard Library](https://docs.python.org/3/library/).
1. Your code must run without errors. Use at least `numpy` 1.15.4. Any code that cannot run will not be graded.
1. Write your own code. Cheating will not be tolerated.
1. Submission includes a zip file that contains this notebook, with your ID as the file name. For example, `hw1_123456789_987654321.zip` if you submitted in pairs and `hw1_123456789.zip` if you submitted the exercise alone. The name of the notebook should follow the same structure.
   
Please use only a **zip** file in your submission.

---
##❗❗❗❗❗❗❗❗❗**This is mandatory**❗❗❗❗❗❗❗❗❗
## Please write your RUNI emails in this cell:

### *** YOUR EMAILS HERE ***

---

## Please sign that you have read and understood the instructions:

### *** YOUR IDS HERE ***

---


In [ ]:
# Import necessary libraries
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances as p_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.manifold import trustworthiness
import numba as nb
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import trange

np.random.seed(42)

# Design your algorithm
Make sure to describe the algorithm, its limitations, and describe use-cases.

The algorithm t-SNE is a dimensionality reduction technique used to visualize high-dimensional data in 2D or 3D and can also be used in fields such as bioinformatics. It models pairwise similarities in high-dimensional space using Gaussians and maps them to a lower-dimensional space using a t-distribution. The algorithm minimizes KL divergence between the two distributions to preserve local structures. Its main goal is to keep similar data points close together in the lower-dimensional space.

The t-SNE algorithm relies on pairwise distances and an iterative process where gradients are computed to minimize the KL divergence. At each step, our implementation of the algorithm updates the positions of the points in the low-dimensional space, using momentum-based gradient descent to ensure efficient optimization. A key part of the algorithm is selecting the right hyperparameters, especially the perplexity, which controls the number of nearest neighbors that influence the probability distributions as well as the learning rate. Also, we initialized the Y values using PCA to ensure faster convergence and reduce the risk of local minima opposed to using random values. Moreover, choosing an appropriate momentum helps the algorithm converge faster.

However, t-SNE has some limitations; it is computationally expensive and does not scale well to very large datasets, as it has a time complexity that is approximately quadratic in the number of data points. Also, it can be sensitive to the choice of hyperparameters, and the results can vary significantly based on the initialization of the low-dimensional representation. Additionally, t-SNE does not preserve global structures well, which can lead to misleading interpretations if not used carefully.


# Your implementations
You may add new cells, write helper functions or test code as you see fit.
Please use the cell below and include a description of your implementation.
Explain code design consideration, algorithmic choices and any other details you think is relevant to understanding your implementation.
Failing to explain your code will lead to point deductions.

In this implementation of t-SNE, we have followed the core principles of the algorithm. fit_transform() is the main method of the algorithm, responsible for the optimization process. Inside this method, the pairwise affinities are computed for the high-dimensional space using compute_pairwise_affinities(), which calculates Gaussian-based affinities based on pairwise distances. This helps in setting up the high-dimensional probability distribution P{ij} for each data point.

After the high-dimensional affinities are computed, we proceed with an iterative optimization procedure. For each iteration, the pairwise affinities in the low-dimensional space are computed using the compute_pairwise_t_distribution_affinities() method, which uses t-distribution. The gradient we implemented is momentum-based and the momentum term helps speed-up convergence, adjusting its value based on whether the iteration has reached the momentum_switch_iter.

We used binary search in binary_search_precision() to determine the optimal gaussian_precision (standard deviation) value, which controls the precision of the Gaussian distribution in the high-dimensional space. This search ensures that the target entropy (determined by the perplexity) is achieved.

The code structure makes sure to handle the computational complexity efficiently by vectorizing operations (numpy) where possible and avoiding unnecessary recomputations.

In [ ]:
class CustomTSNE:
    def __init__(self, perplexity=30.0, n_components=2, n_iter=1000, learning_rate=200.0,
                 momentum_initial=0.5, momentum_final=0.9, momentum_switch_iter=250, tolerance=1e-5, epsilon=1e-10):
        self.perplexity = perplexity
        self.n_components = n_components
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.momentum_initial = momentum_initial
        self.momentum_final = momentum_final
        self.momentum_switch_iter = momentum_switch_iter
        self.tolerance = tolerance
        self.epsilon = epsilon

    # Part 1: Implementing t-SNE
        # Step 1: Compute pairwise affinities in the original space with a Gaussian distribution

    def fit_transform(self, X):
        """
        t-SNE algorithm implementation
        :param X: data points in high-dimensional space
        :return Y: 2D representation of the data points
        """
        n_samples = X.shape[0]
        p_affinities = self._compute_pairwise_affinities(X)

        # random initialization of Y
        # Y = np.random.randn(n_samples, 2)

        # PCA initialization + normalization
        pca = PCA(n_components=self.n_components)
        Y = pca.fit_transform(X)
        min_y = np.min(Y)
        max_y = np.max(Y)
        Y = (Y - min_y) / np.maximum(max_y - min_y, self.epsilon)
        V = np.zeros_like(Y)

        # early stopping
        current_kl = np.inf
        current_iter = 0

        # momentum gradient descent process
        with trange(self.n_iter, desc="t-SNE Optimization") as progress:
            for i in progress:
                q_affinities, inverse_pairwise_distances = self._compute_pairwise_t_distribution_affinities(Y)
                kl = np.sum(p_affinities * np.log2(np.maximum(p_affinities, self.epsilon) / np.maximum(q_affinities, self.epsilon)))

                progress.set_postfix({"KL Divergence": f"{kl:.4f}"})

                pq_affinities = p_affinities - q_affinities

                gradient_Y = np.zeros_like(Y)
                for j in range(n_samples):
                    # compute gradient for each point
                    gradient_Y[j] = 4 * np.sum(
                        pq_affinities[j, :, None] *
                        inverse_pairwise_distances[j, :, None] *
                        (Y[j] - Y), axis=0)

                # update momentum
                if i < self.momentum_switch_iter:
                    momentum = self.momentum_initial
                else:
                    momentum = self.momentum_final

                # update Y with momentum
                V = momentum * V - self.learning_rate * gradient_Y
                Y += V

                if self.tolerance < current_kl - kl:
                    current_kl = kl
                    current_iter = i
                elif i - current_iter > self.momentum_switch_iter:
                    print(f"Early stop at: {i}, KL divergence: {current_kl:.3f}")
                    break

        return Y

    def _compute_pairwise_t_distribution_affinities(self, Y):
        """
        compute pairwise affinities in the low-dimensional space using t-distribution
        :param Y: data points in low-dimensional space
        :return q_affinities: pairwise affinities in low-dimensional space
        :return inverse_pairwise_distances: pairwise distances in low-dimensional space
        """
        # pairwise distances
        distances = p_distances(Y, squared=True)
        inverse_distances = 1 / (1 + distances)

        # set diagonal to 0 for self-affinity
        np.fill_diagonal(inverse_distances, 0)

        # compute q affinities for t-distribution
        q_affinities = np.maximum(inverse_distances / np.sum(inverse_distances), self.epsilon)

        return q_affinities, inverse_distances

    def _compute_pairwise_affinities(self, X):
        """
        compute pairwise affinities in the original space using Gaussian distribution
        :param X: data points in high-dimensional space
        :return p_affinities: pairwise affinities in high-dimensional space
        """
        # pairwise distances
        distances = p_distances(X, squared=True)
        np.fill_diagonal(distances, 0)

        # init pairwise affinities
        n_samples = X.shape[0]
        p_affinities = np.zeros((n_samples, n_samples))

        # entropy based on perplexity
        target_entropy = np.log2(self.perplexity)

        # compute affinities for each data point
        for i in range(n_samples):
            p_i = self._binary_search_precision(distances, target_entropy, i)
            p_affinities[i] = p_i
            p_affinities[i, i] = 0 # zero out the self-affinity

        # symmetric SNE
        p_affinities = (p_affinities + p_affinities.T) / (2 * n_samples)

        return p_affinities

    def _binary_search_precision(self, distances, target_entropy, i):
        """
        binary search to find the correct precision for Gaussian distribution
        :param distances: pairwise distances
        :param target_entropy: target entropy based on perplexity
        :param i: index of the data point
        :return p_affinities: pairwise affinities for the data point
        """
        precision = 1.0
        min_precision, max_precision = -np.inf, np.inf
        p_affinities = np.zeros_like(distances[i])

        # binary search to find the correct precision
        for j in range(50):
            # compute affinities for point i
            p_affinities = np.exp(-distances[i] / (2 * precision ** 2))
            p_affinities[i] = 0  # zero out the self-affinity

            p_sum = np.sum(p_affinities)
            if p_sum == 0:
                p_sum = self.epsilon
            p_affinities /= p_sum

            # similarity in high-dimensional space
            entropy = -np.sum(p_affinities * np.log2(p_affinities + self.epsilon))

            # convergence check
            if np.abs(entropy - target_entropy) < self.tolerance:
                break

            # adjust precision based on entropy
            if entropy < target_entropy:
                min_precision = precision
                precision = (min_precision + max_precision) / 2 if max_precision != np.inf else precision * 2
            else:
                max_precision = precision
                precision = (max_precision + min_precision) / 2 if min_precision != 0 else precision / 2

        return p_affinities

    def transform(self, X_original, Y_original, X_new):
        """
        Transform new data points into the low-dimensional space using the learned mapping
        :param X_original:
        :param Y_original:
        :param X_new:
        :return: newly transformed data points in low-dimensional space
        """
        # use nearest neighbors to find the closest points in the original space
        n_neighbors = max(int(self.perplexity), 2)
        knn = NearestNeighbors(n_neighbors=n_neighbors)
        knn.fit(X_original)

        # calculate distances and indices of neighbors
        distances, indices = knn.kneighbors(X_new)
        precision = np.std(distances, axis=1, keepdims=True) + self.epsilon

        # compute weights based on Gaussian distribution
        weights = np.exp(-distances ** 2 / (2 * precision ** 2))
        row_sums = np.sum(weights, axis=1, keepdims=True)
        row_sums[row_sums == 0] = 1  # prevent division by zero
        weights /= row_sums

        # compute the new low-dimensional representation
        Y_neighbors = Y_original[indices]
        Y_new = np.einsum('ij,ijk->ik', weights, Y_neighbors)

        return Y_new

    def neighborhood_preservation(self, X_train, Y_train, X_new, Y_new):
        """
        check the neighborhood preservation of the learned mapping
        :param X_train:
        :param Y_train:
        :param X_new:
        :param Y_new:
        :return: neighborhood preservation score
        """
        # find kNN in high-dimensional space
        knn_high = NearestNeighbors(n_neighbors=max(int(self.perplexity), 2))
        knn_high.fit(X_train)
        neighbors_high = knn_high.kneighbors(X_new, return_distance=False)

        # find kNN in low-dimensional space
        knn_low = NearestNeighbors(n_neighbors=max(int(self.perplexity), 2))
        knn_low.fit(Y_train)
        neighbors_low = knn_low.kneighbors(Y_new, return_distance=False)

        total = 0
        for high, low in zip(neighbors_high, neighbors_low):
            intersection = len(set(high) & set(low))
            total += intersection / max(int(self.perplexity), 2)

        nps = total / X_new.shape[0]

        return nps

# Load data
Please use the cell below to discuss your dataset choice and why it is appropriate (or not) for this algorithm.

We choose to use the MNIST dataset which is appropriate for t-SNE because it contains high-dimensional image data (784 features per sample), where visualizing the raw space is impossible. MNIST has a clear class structure which is ideal for observing how well t-SNE preserves local neighborhoods. Each image represents a handwritten digit (0–9), and t-SNE helps reduce this complex space into 2D while preserving local similarities. We are using a portion of the dataset (2000 samples) as tsne works better with smaller subsets of data. We experimented with various dataset sizes and we found that 2000 samples is balanced in terms of visual result and runtime efficiency. For example, with 500 samples we the results fast and saw similar separation for different clusters, but the distances inside each cluster are more significant. On the other hand, we saw that with 10,000 samples, for example, it takes a significant time to run, but the clusters are visually more dense. Furthermore, each class in MNIST is relatively balanced, but certain digits (e.g. 1 and 7) may overlap, which helps testing t-SNE’s ability to separate visually similar classes. Using PCA before t-SNE already takes care of normalizing and scaling and so normalizing the data beforehand does not affect the results.

Behavior on MNIST (2000 samples):
- t-SNE often reveals tight clusters for digits like 0, 1, and 6
- Some classes (for example 4 and 9 or 5 and 3) show partial overlap, reflecting visual similarity and t-SNE's focus on local, not global, structure.

In summary, MNIST enables to demonstrate how well t-SNE captures local structure in complex data.

In [ ]:
# Load data
print("Loading MNIST dataset...")
mnist = fetch_openml("mnist_784", as_frame=False)  # version=1)
X = mnist.data
Y = mnist.target

# Split the data into train and test
print("Splitting data...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train_subset = X_train[:2000]
Y_train_subset = Y_train[:2000]
x_test_subset = X_test[:2000]
Y_test_subset = Y_test[:2000]

# t-SNE demonstration
Demonstrate your t-SNE implementation.

Add plots and figures. The code below is just to help you get started, and should not be your final submission.

Please use the cell below to describe your results and tests.

Describe the difference between your implementation and the sklearn implementation. Hint: you can look at the documentation.

In this demonstration, we applied our custom t-SNE implementation and compared it against the standard sklearn t-SNE on a subset of 2000 MNIST samples. To ensure a fair comparison, we performed a hyperparameter search over different perplexity values (5 to 50) as well as learning rate values (100 to 500), number of iterations (850 found to be best balance) and evaluated the quality of the embeddings using the nearest neighbor preservation (trustworthiness) metric. We found that perplexity values around 35 and learning rate around 250 yielded the best trustworthiness scores for both the custom and sklearn versions, indicating that the local neighborhood structure was well-preserved in the low-dimensional space.

Generally, both implementations behaved similarly, but there are some differences. Our custom t-SNE manually handles the affinity computation, gradient updates, and optimization process with PCA-based initialization, while sklearn’s t-SNE includes more internal optimizations such as adaptive learning rates and Barnes-Hut optimization. As a result, sklearn’s version is typically more stable and faster for large datasets. However, after tuning the perplexity and the learning rate, our custom t-SNE achieved comparable neighborhood preservation and produced visually similar 2D embeddings.

Visual description of the results:
The custom t-SNE separates well the MNIST digits, with clear and tight clusters, but the groups are a bit closer together and sometimes slightly overlap. In comparison, the sklearn t-SNE spreads the clusters out more naturally, leaving bigger gaps between different digits and creating more clear shapes. Both versions successed in separating the digits, but sklearn’s version looks cleaner overall, mainly because of small internal optimizations like early exaggeration tuning (as described in the documentation). Overall, our custom t-SNE is working well and with some more small modifications we could get even closer to the sklearn output.

In [ ]:
# Run your custom t-SNE implementation
custom_tsne = CustomTSNE(n_components=2, perplexity=30, learning_rate=300, n_iter=850)
custom_Y = custom_tsne.fit_transform(X_train_subset)

# Run sklearn t-SNE
sk_tsne = TSNE(n_components=2, init='pca', perplexity=30)
sk_Y = sk_tsne.fit_transform(X_train_subset)

# Visualization of the result
plt.figure()
plt.scatter(custom_Y[:, 0], custom_Y[:, 1], s=5, c=Y_train_subset.astype(int), cmap='tab10')
plt.scatter(custom_Y[:, 0], custom_Y[:, 1], s=5, c=Y_train_subset.astype(int), cmap='tab10')
plt.colorbar()
plt.title('MNIST Data Embedded into 2D with Custom t-SNE')

plt.figure()
plt.scatter(sk_Y[:, 0], sk_Y[:, 1], s=5, c=Y_train_subset.astype(int), cmap='tab10')
plt.colorbar()
plt.title('MNIST Data Embedded into 2D with sklearn t-SNE')
plt.show()

# optimizing the hyperparameters values (uncomment to run optimization)

# perplexities = list(range(25, 45, 5))
# learning_rates = list(range(200, 400, 50))
#
# trust_scores = np.zeros((len(learning_rates), len(perplexities)))
#
# # running all combinations of perpelexity and learning rate values
# for i, lr in enumerate(learning_rates):
#     for j, p in enumerate(perplexities):
#         print(f"Learning rate = {lr}, Perplexity = {p}")
#         custom_tsne = CustomTSNE(n_components=2, perplexity=p, learning_rate=lr, n_iter=850)
#         Y_custom = custom_tsne.fit_transform(X_train_subset)
#         trust_cust = trustworthiness(X_train_subset, Y_custom, n_neighbors=5)
#         trust_scores[i, j] = trust_cust
#         print(f"Trustworthiness = {trust_cust:.4f}")
#
# # plotting the results
# plt.figure(figsize=(10, 7))
# plt.imshow(trust_scores, interpolation='nearest', cmap='viridis')
# plt.colorbar(label='Trustworthiness')
# plt.xticks(ticks=np.arange(len(perplexities)), labels=perplexities)
# plt.yticks(ticks=np.arange(len(learning_rates)), labels=learning_rates)
# plt.xlabel('Perplexity')
# plt.ylabel('Learning Rate')
# plt.title('Trustworthiness Heatmap (Custom t-SNE)')
# plt.tight_layout()
#
# for i in range(len(learning_rates)):
#     for j in range(len(perplexities)):
#         text = f"{trust_scores[i, j]:.3f}"
#         plt.text(j, i, text, ha='center', va='center', color='white', fontsize=8)
#
# plt.show()
#
# best_idx = np.unravel_index(np.argmax(trust_scores), trust_scores.shape)
# best_lr = learning_rates[best_idx[0]]
# best_perplexity = perplexities[best_idx[1]]
# best_trustworthiness = trust_scores[best_idx]
#
# print(f"\nBest combination:")
# print(f"Perplexity = {best_perplexity}")
# print(f"Learning Rate = {best_lr}")
# print(f"Trustworthiness = {best_trustworthiness:.4f}")

# t-SNE extension - mapping new samples
Demonstrate your t-SNE transformation procedure.

Add plots and figures.

Please use the cell below t describe your suggested approach in detail. Use formal notations where appropriate.
Describe and discuss your results.

In order to transform new high-dimensional points into the learned low-dimensional space, we used a local interpolation approach based on nearest neighbors.

Given a new point $x_{\text{new}} \in \mathbb{R}^D$:

1. Neighbor search: We identified its k-nearest neighbors in the original high-dimensional space $X_{\text{train}}$ using Euclidean distance, where $k = perplexity$.
2. Local scale estimation: We computed the standard deviation $\sigma$ of the distances between $x_{\text{new}}$ and its neighbors to capture the local neighborhood scale:
$\sigma = \text{std}\left(\{ \|x_{\text{new}} - x_i\| \}_{i=1}^k\right) + \epsilon$
where $\epsilon$ is a small constant to prevent division by zero.
3. Weight calculation:
We assigned a weight to each neighbor based on a Gaussian kernel applied to the squared distances:
$w_i = \exp\left( -\frac{\| x_{\text{new}} - x_i \|^2}{2\sigma^2} \right)$
The weights were then normalized:
$\frac{w_i}{\sum_j w_j}$
4. Low-dimensional interpolation:
The final embedding $y_{\text{new}} \in \mathbb{R}^2$ was computed as the weighted sum of the corresponding neighbor embeddings in the low-dimensional space $Y_{\text{train}}$:
$y_{\text{new}} = \sum_{i=1}^k w_i \, y_{\text{train},i}$

From the visualizations, we observe that the transformed test points generally align well with the corresponding training clusters, suggesting that the local interpolation method is effective. However, small deviations and slight misplacements occur in some classes, particularly for more overlapping digits like '2', '9' reflecting the challenge of preserving neighborhood structures exactly after projection.

Since the goal of the transformation is to map new data points into the low-dimensional space while preserving local neighborhood relationships, the performance measure we propose is the Neighborhood Preservation Score (NPS).

The NPS is defined as the ratio of the number of correctly preserved neighbors in the low-dimensional space to the total number of neighbors in the high-dimensional space. The value of NPS ranges from 0 to 1, where:
- $1$ = perfect preservation of neighborhood structure
- $0$ = no local structure preserved

Similar NPS scores between our custom t-SNE(0.4245) and sklearn(0.4194) in both training and testing suggest that our extension preserves local relationships well, making the method effective for transforming new data points without retraining t-SNE.

In [ ]:
# Transform new data
custom_Y_new = custom_tsne.transform(X_train_subset, custom_Y, x_test_subset)
# custom_Y_sk = custom_tsne.transform(X_train_subset, sk_Y, x_test_subset)

# Visualization of the result
plt.figure()
plt.scatter(custom_Y[:, 0], custom_Y[:, 1], s=5, c=Y_train_subset.astype(int), cmap='tab10')
plt.scatter(custom_Y_new[:, 0], custom_Y_new[:, 1], marker = '*', s=50, linewidths=0.5, edgecolors='k', c=Y_test_subset.astype(int), cmap='tab10')
plt.colorbar()
plt.title('MNIST Data Embedded into 2D with Custom t-SNE')

# Visualization of the result for each label
fig, axes = plt.subplots(5, 2, figsize=(16, 40))  # 5 rows, 2 columns, much taller figure
axes = axes.flatten()

for label in range(10):
    ax = axes[label]
    ax.scatter(custom_Y[Y_train_subset == str(label), 0], custom_Y[Y_train_subset == str(label), 1], s=5, c=Y_train_subset[Y_train_subset == str(label)].astype(int), cmap='tab10', vmin=0, vmax=9, label='Train')
    ax.scatter(custom_Y_new[Y_test_subset == str(label), 0], custom_Y_new[Y_test_subset == str(label), 1], marker='*', s=50, linewidths=0.5, edgecolors='k', c=Y_test_subset[Y_test_subset == str(label)].astype(int), cmap='tab10', vmin=0, vmax=9, label='Test')
    ax.set_title(f'Label {label}', fontsize=14)
    ax.legend(fontsize=10)

plt.tight_layout()
plt.show()

# check the neighborhood preservation score (NPS)

# nps_score = custom_tsne.neighborhood_preservation(X_train_subset, custom_Y, x_test_subset, custom_Y_new)
# print(f"Neighborhood Preservation Score: {nps_score:.4f}")
# nps_score_sk = custom_tsne.neighborhood_preservation(X_train_subset, sk_Y, x_test_subset, custom_Y_sk)
# print(f"Neighborhood Preservation Score (sklearn): {nps_score_sk:.4f}")

# Use of generative AI
Please use the cell below to describe your use of generative AI in this assignment.

We used generative AI tools such as ChatGPT and Claude to assist with technical aspects like using Python packages, improving code readability, and debugging issues. We also used them for creating visualizations and making the code more efficient. For example, helping us to find the right NumPy operations to optimize performance. The core algorithm design, theoretical understanding, data processing, and full implementation were based on material taught in class and additional study from online sources.